In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2021-11-21 00:30:52.850483: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2021-11-21 00:31:02.303062: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-21 00:31:02.342144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-21 00:31:02.651130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 00:31:02.651722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.6

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-11-21 00:31:09.292742: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-21 00:31:09.292816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 00:31:09.293279: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-11-21 00:31:09.293326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 00:31:09.293775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:04:00

In [3]:
import numpy as np
import pandas as pd

In [4]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [5]:
# 顯示所有列
pd.set_option('display.max_columns', None)

In [6]:
# 讀前處理3做完留每個月消費次數資料最多的data
train = pd.read_csv('Processed_tbrain_training_groupby_txn_amt.csv')
print(train.shape)
print(train.info())
train.head(5)

(8948290, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8948290 entries, 0 to 8948289
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  int64  
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_txn

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10000000,10,3,1.684984e-111,0,3,0,0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
1,2,10000000,46,1,2.805564e-111,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
2,3,10000000,10,3,1.719186e-111,0,3,0,0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
3,4,10000000,10,2,1.220472e-111,0,2,0,0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
4,5,10000000,10,7,2.045684e-111,0,7,0,0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1


In [7]:
# 特徵選擇
train.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty

In [8]:
# # 先全丟
# SelectedFeatures = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
#        'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
#        'domestic_offline_amt_pct', 'domestic_online_amt_pct',
#        'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
#        'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
#        'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
#        'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
#        'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
#        'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
#        'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
#        'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
#        'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
#        'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
#        'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
#        'gender_code', 'age', 'primary_card']

In [9]:
# 先不管消費者其他消費資料
SelectedFeatures = ['dt', 'chid', 'shop_tag',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
       'gender_code', 'age', 'primary_card']

In [10]:
train = train[SelectedFeatures]
train.head(5)

,dt,chid,shop_tag,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10000000,10,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
1,2,10000000,46,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
2,3,10000000,10,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
3,4,10000000,10,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
4,5,10000000,10,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1


In [11]:
# 先取出每個顧客不重複最後一個月的值之後預測用
pre_train = train.drop_duplicates(subset=['chid'], keep='last')
pre_train

,dt,chid,shop_tag,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
23,24,10000000,10,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
45,24,10000001,21,2.0,4.0,11.0,1.0,3.0,30.0,8.670854e-214,0.0,5.0,1
51,22,10000002,22,2.0,4.0,13.0,1.0,99.0,30.0,1.129543e-213,0.0,3.0,1
53,21,10000003,37,2.0,6.0,15.0,1.0,99.0,30.0,1.129543e-213,0.0,6.0,1
68,18,10000004,31,2.0,5.0,15.0,1.0,99.0,30.0,6.369410e-214,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8948210,19,10499995,10,1.0,3.0,5.0,1.0,99.0,30.0,1.258009e-213,1.0,4.0,1
8948231,24,10499996,36,2.0,3.0,11.0,1.0,2.0,30.0,7.182541e-214,1.0,3.0,1
8948253,24,10499997,36,1.0,6.0,8.0,1.0,2.0,30.0,1.017454e-213,0.0,3.0,1
8948277,24,10499998,10,2.0,6.0,13.0,1.0,99.0,30.0,9.594880e-214,1.0,4.0,1


In [12]:
# 去除shop_tag
pre_train.drop(columns = ['shop_tag'], axis=1, inplace=True)

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [14]:
# 按照要提交的檔案排序
pre_train = pre_train.set_index('chid', drop= False)
pre_train = pre_train.loc[predict.chid]
pre_train.head(5)

,dt,chid,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,
10128239,24,10128239,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
10077943,24,10077943,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1
10277876,24,10277876,1.0,5.0,6.0,1.0,1.0,30.0,7.077406e-214,0.0,3.0,1
10364842,24,10364842,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1
10392717,24,10392717,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1


In [15]:
# # 把chid全部拿掉
# pre_train.drop(columns = ['chid'], axis=1, inplace=True)
# pre_train

In [16]:
# train.drop(columns = ['chid'], axis=1, inplace=True)
# train

In [17]:
# 看類別不平均的問題
train.shop_tag.value_counts()

37    1715329
10     763810
0      694846
2      669107
45     530365
36     485187
48     458509
15     424685
16     394608
19     273574
12     261142
39     183114
6      163980
46     156816
26     134084
17     131778
18     127780
34     122944
11     104130
41      95678
25      95316
35      92177
31      89076
22      86402
21      85278
13      78644
20      74072
43      56729
9       51076
33      42018
38      40032
40      27499
23      27255
5       26742
32      23337
3       23331
47      22174
8       19442
30      16598
1       15816
24      14525
7       10983
14       9998
44       9965
42       8491
28       4353
4        3786
27       1683
29         26
Name: shop_tag, dtype: int64

In [18]:
# 拆分測試訓練集
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [19]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(7158632, 12)
(7158632, 1)


In [20]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(1789658, 12)
(1789658, 1)


In [21]:
import gc
del train
del test
gc.collect()

0

In [22]:
X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train1.shape

(7158632, 12, 1)

In [23]:
y_train1 = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))
y_train1.shape

(7158632, 1, 1)

In [24]:
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test1.shape

(1789658, 12, 1)

In [25]:
y_test1 = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
y_test1.shape

(1789658, 1, 1)

In [26]:
pre_train = pre_train.values
pre_train = np.reshape(pre_train, (pre_train.shape[0], pre_train.shape[1], 1))
pre_train.shape

(500000, 12, 1)

In [27]:
# 調類別權重
from sklearn.utils import compute_class_weight
from sklearn.utils import class_weight

train_classes = pd.DataFrame(np.array(y_train))


class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_classes), class_weights)),
class_weights

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


({0: 0.2626640464223402,
  1: 11.51620137255596,
  2: 0.27289739834096993,
  3: 7.8150492464023165,
  4: 49.20664554134217,
  5: 6.8325942667778925,
  6: 1.1144342612667717,
  7: 16.70606410660319,
  8: 9.403007698541861,
  9: 3.5832073631964314,
  10: 0.23907946967250107,
  11: 1.7503478136276436,
  12: 0.7006327061080813,
  13: 2.319918229940053,
  14: 18.207194742303702,
  15: 0.4302085768492738,
  16: 0.46282976861524416,
  17: 1.3858594415777656,
  18: 1.4295103730197447,
  19: 0.6686064941568877,
  20: 2.461451495497193,
  21: 2.14283978134068,
  22: 2.1120472244874358,
  23: 6.737745266441217,
  24: 12.498462709576945,
  25: 1.9209568407853064,
  26: 1.3609178445481593,
  27: 112.2077808081758,
  28: 42.11430689312335,
  29: 7304.7265306122445,
  30: 11.011873868413726,
  31: 2.0520335783727073,
  32: 7.798779192454219,
  33: 4.3378523890924585,
  34: 1.4842781587784462,
  35: 1.984467740831102,
  36: 0.3759896299470993,
  37: 0.1064426289269911,
  38: 4.57531961455153,
  39: 0.

In [28]:
from keras import backend as K
def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):

        axis = -1 #if channels last 
        #axis=  1 #if channels first


        #argmax returns the index of the element with the greatest value
        #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
        classSelectors = tf.cast(classSelectors, tf.int32)
            #if your loss is sparse, use only true as classSelectors

        #considering weights are ordered by class, for each class
        #true(1) if the class index is equal to the weight index   
        classSelectors = [K.equal(i, classSelectors) for i in range(len(weightsList))]

        #casting boolean to float for calculations  
        #each tensor in the list contains 1 where ground true class is equal to its index 
        #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

        #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        #sums all the selections
        #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


        #make sure your originalLossFunc only collapses the class axis
        #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

Using TensorFlow backend.


In [29]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))
model.compile(loss= weightedLoss(tf.keras.losses.sparse_categorical_crossentropy, class_weights[0]), optimizer='adam', metrics = ['accuracy'])
model.summary()

2021-11-21 00:31:41.905777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 00:31:41.931532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-11-21 00:31:41.931850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 00:31:41.933722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 12, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 49)                2499      
Total params: 33,099
Trainable params: 33,099
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(X_train1, y_train1, epochs=1, batch_size=1, verbose=1)

2021-11-21 00:31:43.596204: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 343614336 exceeds 10% of free system memory.
2021-11-21 00:31:43.777376: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-21 00:31:43.915973: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3199980000 Hz


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-11-21 00:31:47.660023: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-11-21 00:31:48.920183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


7158632/7158632 [==============================] - 17978s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0088


In [31]:
# 模型預測
predictions = model.predict(pre_train)

In [32]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

array([[13, 46, 15, ...,  4, 31,  1],
       [13, 15, 46, ...,  4, 31,  1],
       [13, 46, 15, ...,  4, 31,  1],
       ...,
       [13, 46, 15, ...,  4, 31,  1],
       [13, 15, 46, ...,  4, 31,  1],
       [13, 15, 46, ...,  4, 31,  1]])

In [33]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [34]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [35]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [36]:
filename = 'submission-砍掉卡片相關的欄位訓練一次-用groupby_txn_amt全部資料-有調權重-不砍chid.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

            chid  top1  top2  top3
0       10128239    13    15     6
1       10077943    13    15     6
2       10277876    13    15     6
3       10364842    13    15     6
4       10392717    13    15     6
...          ...   ...   ...   ...
499995  10433239    13    15     6
499996  10235119    13    15     6
499997  10267595    13    15     6
499998  10160606    13    15     6
499999  10141759    13    15    12

[500000 rows x 4 columns]
Saved file: submission-砍掉卡片相關的欄位訓練一次-用groupby_txn_amt全部資料-有調權重-不砍chid.csv


In [37]:
# 保存模型
model.save('LSTM_model_all-砍掉卡片相關的欄位訓練一次-用groupby_txn_amt全部資料-有調權重-不砍chid.h5') 